# AULA 55 - EXPLICABILIDADE E ÉTICA Parte 2 (17-10-2025)

Alan Diek Guimarães

## Códigos da aula anterior

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers, models


In [2]:
# Tratar os dados

# Ler os dados
dados = pd.read_csv('churn.csv')

# Preparar o encoding dos dados
encoder = LabelEncoder()
dados['Gender'] = encoder.fit_transform(dados['Gender'])
dados['Subscription Type'] = encoder.fit_transform(dados['Subscription Type'])
dados['Contract Length'] = encoder.fit_transform(dados['Contract Length'])

# Separar o X e o y
X = dados.drop('Churn', axis=1).values
y = dados['Churn'].values

# Normalizar os dados
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
from tensorflow.keras.models import load_model

# Carregando o modelo
modelo2 = load_model('modelo_churn_(Alan_Diek).h5')

In [4]:
exemplo = dados.drop('Churn', axis=1).iloc[0].values

resultado_esperado = dados['Churn'].iloc[0]
entrada = np.array(exemplo).reshape(1, -1)

predicao = modelo2.predict(entrada)

print(f'\nResultado esperado: {resultado_esperado}')
print(f'\nPredição do modelo: {predicao[0][0]:.2f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step

Resultado esperado: 1

Predição do modelo: 1.00


In [5]:
modelo2.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 32)             │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 931 (3.64 KB)

 Trainable params: 929 (3.63 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

## Continuação

Construindo o modelo ***SHAP***

Para usar o SHAP precisamos ter um modelo e um conjunto de dados

In [7]:
# instalar a biblioteca shap (executar no kernel do notebook)
%pip install shap

# importar a biblioteca do shap
import shap


# Criar um explicador do modelo com o SHAP
# Criar um KernelExplainer usando uma amostra de treinamento

explainer = shap.KernelExplainer(
    lambda data: modelo2.predict(data).flatten(),
    shap.sample(X_train, 50)
)
shap_values = explainer.shap_values(X_train[:100])  # Calcular os valores SHAP para os primeiros 100 exemplos de treinamento

print("Calculando o SHAP, shap value shape:", shap_values[0].shape)


Note: you may need to restart the kernel to use updated packages.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


  0%|          | 0/100 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
3197/3197 ━━━━━━━━━━━━━━━━━━━━ 1s 401us/step


  1%|          | 1/100 [00:01<03:07,  1.89s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
3197/3197 ━━━━━━━━━━━━━━━━━━━━ 1s 408us/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
3197/3197 ━━━━━━━━━━━━━━━━━━━━ 1s 382us/step


 99%|█████████▉| 99/100 [03:00<00:01,  1.83s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
3197/3197 ━━━━━━━━━━━━━━━━━━━━ 1s 385us/step


100%|██████████| 100/100 [03:02<00:00,  1.83s/it]

Calculando o SHAP, shap value shape: (11,)
